In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
from digitModel import Model
from datasetClass import MNISTDataset
import time
import copy


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
#Loading Data
df_train = pd.read_csv('digit-recognizer/train.csv')
df_test = pd.read_csv('digit-recognizer/test.csv')

In [3]:
#Preparing std and mean for image preprocessing later to improve performance
train_data = df_train.drop('label', axis=1).values
train_mean = train_data.mean()/255.
train_std = train_data.std()/255.

In [4]:
mask = np.random.rand(len(df_train)) < 0.8
df_val = df_train[~mask]
df_train = df_train[mask]

In [5]:
batch_size = 128
classes = range(10)


train_transform = transforms.Compose(
                    [
                    transforms.ToPILImage(),
                    transforms.RandomRotation(30),
                    transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                    transforms.GaussianBlur(kernel_size = 3, sigma=(0.1, 2.0)),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[train_mean], std=[train_std]),
                    ])


val_transform = transforms.Compose(
                    [
                    transforms.ToPILImage(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[train_mean], std=[train_std]),
                    ])
test_transform = val_transform

train_dataset = MNISTDataset(df_train, transform = train_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                batch_size=batch_size,shuffle = True)
val_dataset = MNISTDataset(df_val, transform = val_transform)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                batch_size=batch_size,shuffle = False)

In [6]:
model = Model()
model.to(device)
criterion = nn.NLLLoss()   # with log_softmax() as the last layer, this is equivalent to cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [7]:
epochs = 10
train_losses, val_losses = [], []
train_accu, val_accu = [], []
start_time = time.time()
early_stop_counter = 10   # stop when the validation loss does not improve for 10 iterations to prevent overfitting
counter = 0
best_val_loss = float('Inf')

In [8]:
for e in range(epochs):
    epoch_start_time = time.time()
    running_loss = 0
    accuracy=0
    # training step
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        log_ps = model(images)
        
        ps = torch.exp(log_ps)                
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
      
    # record training loss and accuracy
    train_losses.append(running_loss/len(train_loader))
    train_accu.append(accuracy/len(train_loader))
    
    
    #validation step
    val_loss = 0
    accuracy=0
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            log_ps = model(images)
            val_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
            
    # record validation loss and accuracy
    val_losses.append(val_loss/len(val_loader))
    val_accu.append(accuracy/len(val_loader))
    
    print("Epoch: {}/{}.. ".format(e+1, epochs),
          "Time: {:.2f}s..".format(time.time()-epoch_start_time),
          "Training Loss: {:.3f}.. ".format(train_losses[-1]),
          "Training Accu: {:.3f}.. ".format(train_accu[-1]),
          "Val Loss: {:.3f}.. ".format(val_losses[-1]),
          "Val Accu: {:.3f}".format(val_accu[-1])
         )

    
    #Get best run and stop if training does not improve (early_stop_counter == patience)
    if val_losses[-1] < best_val_loss:
        best_val_loss = val_losses[-1]
        counter=0
        best_model_wts = copy.deepcopy(model.state_dict())
    else:
        counter+=1
        print('Validation loss has not improved since: {:.3f}..'.format(best_val_loss), 'Count: ', str(counter))
        if counter >= early_stop_counter:
            print('Early Stopping Now!!!!')
            model.load_state_dict(best_model_wts)
            break
        

Epoch: 1/10..  Time: 146.06s.. Training Loss: 2.211..  Training Accu: 0.188..  Val Loss: 1.615..  Val Accu: 0.672
Epoch: 2/10..  Time: 128.24s.. Training Loss: 1.872..  Training Accu: 0.349..  Val Loss: 1.002..  Val Accu: 0.808
Epoch: 3/10..  Time: 123.04s.. Training Loss: 1.660..  Training Accu: 0.430..  Val Loss: 0.750..  Val Accu: 0.858
Epoch: 4/10..  Time: 140.36s.. Training Loss: 1.523..  Training Accu: 0.489..  Val Loss: 0.586..  Val Accu: 0.885
Epoch: 5/10..  Time: 135.51s.. Training Loss: 1.403..  Training Accu: 0.529..  Val Loss: 0.471..  Val Accu: 0.907
Epoch: 6/10..  Time: 123.80s.. Training Loss: 1.319..  Training Accu: 0.567..  Val Loss: 0.402..  Val Accu: 0.919
Epoch: 7/10..  Time: 111.57s.. Training Loss: 1.246..  Training Accu: 0.592..  Val Loss: 0.335..  Val Accu: 0.928
Epoch: 8/10..  Time: 108.49s.. Training Loss: 1.200..  Training Accu: 0.609..  Val Loss: 0.309..  Val Accu: 0.935
Epoch: 9/10..  Time: 120.07s.. Training Loss: 1.131..  Training Accu: 0.632..  Val Loss:

In [9]:
x_test = df_test.values
x_test = x_test.reshape([-1, 28, 28]).astype(np.float64)
x_test = x_test/255.
x_test = (x_test-train_mean)/train_std

In [10]:
x_test = np.expand_dims(x_test, axis=1)
x_test = torch.from_numpy(x_test).float().to(device)
# x_test.shape
x_test.type()

'torch.FloatTensor'

In [11]:
# prediction time!
model.eval()   # this is needed to disable dropouts
with torch.no_grad():    # turn off gradient computation because we don't need it for prediction
    ps = model(x_test)
    prediction = torch.argmax(ps, 1)
    print('Prediction',prediction)

Prediction tensor([2, 0, 9,  ..., 3, 9, 2])


In [ ]:
# plot training history
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
ax = plt.gca()
ax.set_xlim([0, e + 2])
plt.ylabel('Loss')
plt.plot(range(1, e + 2), train_losses[:e+1], 'r', label='Training Loss')
plt.plot(range(1, e + 2), val_losses[:e+1], 'b', label='Validation Loss')
ax.grid(linestyle='-.')
plt.legend()
plt.subplot(2,1,2)
ax = plt.gca()
ax.set_xlim([0, e+2])
plt.ylabel('Accuracy')
plt.plot(range(1, e + 2), train_accu[:e+1], 'r', label='Training Accuracy')
plt.plot(range(1, e + 2), val_accu[:e+1], 'b', label='Validation Accuracy')
ax.grid(linestyle='-.')
plt.legend()
plt.show()